# Storing RNA-seq data with Bioconductor, intro to SummarizedExperiment & SingleCellExperiment

- **Notes: Prior to the rstat journal club, software need to be installed and R should be updated to R4.0.**
- [Google doc with information](https://docs.google.com/document/d/1umDODmdQldf5w2lNDoFe-unmezHPonpCiKD270VwkrQ/edit?usp=sharing)

Most of the notes below are copied from the google doc.

```
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("SummarizedExperiment")
BiocManager::install("rtracklayer")
BiocManager::install("SingleCellExperiment")
```

## RNAseq gene workflow
- [rnaseqGene workflow](http://bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html)
- This document has a good tutorial for proforming differential expression
 - Uses DESeq2
 - Review Salmon
 - Section 3 introduces *SummarizedExperiment*

![summarizedexperiment](figures/huber2015_Fig2.jpg)

> Its assays component is one or several rectangular arrays of equivalent row and column dimensions. Rows correspond to features, and columns to samples. The component rowData stores metadata about the features, including their genomic ranges. The colData component keeps track of sample-level covariate data. The exptData component carries experiment-level information, including MIAME (minimum information about a microarray experiment)-structured metadata<sup>21</sup>. The R expressions exemplify how to access components. For instance, provided that these metadata were recorded, `rowData(se)$entrezId` returns the NCBI Entrez Gene identifiers of the features, and `se$tissue` returns the tissue descriptions for the samples. Range-based operations, such as %in%, act on the rowData to return a logical vector that selects the features lying within the regions specified by the data object CNVs. Together with the bracket operator, such expressions can be used to subset a SummarizedExperiment to a focused set of genes and tissues for downstream analysis. - Huber 2015.

<sup>21</sup>Brazma, A. et al. Minimum information about a microarray experiment (MIAME) - toward standards for microarray data. Nat. Genet. 29, 365–371 (2001).

## SummarizedExperiment

- [Huber 2015](https://www.nature.com/articles/nmeth.3252)
- These Rdata are huge, and do not play well with other programs
- Load times for this depend on coding for R; but I think this is still much to big
- I don't see many advantages to this, then separate files

## Discussion of GenomicRanges

- [Genomic Ranges Documentation](http://bioconductor.org/packages/release/bioc/html/GenomicRanges.html)
- [Genomic Ranges Manuscript](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003118)
- [Introduction to GenomicRanges](http://bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html)

> The GenomicRanges package serves as the foundation for representing genomic locations within the Bioconductor project. In the Bioconductor package hierarchy, it builds upon the IRanges (infrastructure) package and provides support for the BSgenome (infrastructure), Rsamtools (I/O), ShortRead (I/O & QA), rtracklayer (I/O), GenomicFeatures (infrastructure), GenomicAlignments (sequence reads), VariantAnnotation (called variants), and many other Bioconductor packages.\
> This package lays a foundation for genomic analysis by introducing three classes (GRanges, GPos, and GRangesList), which are used to represent genomic ranges, genomic positions, and groups of genomic ranges. This vignette focuses on the GRanges and GRangesList classes and their associated methods.

## rtracklayer - R interface to genome annotation files and the UCSC genome browser

- [rtracklayer Documentation](http://bioconductor.org/packages/release/bioc/html/rtracklayer.html)
- [rtracklayer Vignette](http://bioconductor.org/packages/release/bioc/vignettes/rtracklayer/inst/doc/rtracklayer.pdf)
- [BED file format info](https://genome.ucsc.edu/FAQ/FAQformat.html#format1)
- [GTF file format info](https://useast.ensembl.org/info/website/upload/gff.html)

### Get a GRanges object to work with

In [1]:
library("rtracklayer")
genes_gencode_v26 <- rtracklayer::import( ## read data from a website
    paste0(
        "http://snaptron.cs.jhu.edu/data/", 
        "temp/recount3/human/annotations/",
        "gene_sums/human.gene_sums.G026.gtf.gz"
    )
)
genes_gencode_v26

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vecto

GRanges object with 63856 ranges and 10 metadata columns:
            seqnames            ranges strand |   source     type     score
               <Rle>         <IRanges>  <Rle> | <factor> <factor> <numeric>
      [1] GL000009.2       56140-58376      - |  ENSEMBL     gene      2237
      [2] GL000194.1      53590-115018      - |  ENSEMBL     gene      2179
      [3] GL000194.1      53594-115055      - |  ENSEMBL     gene      1599
      [4] GL000195.1       37434-37534      - |  ENSEMBL     gene       101
      [5] GL000195.1       42939-49164      - |  ENSEMBL     gene      2195
      ...        ...               ...    ... .      ...      ...       ...
  [63852]       chrY 57184101-57197337      + |   HAVANA     gene      2504
  [63853]       chrY 57201143-57203357      - |   HAVANA     gene      1054
  [63854]       chrY 57190738-57208756      + |   HAVANA     gene       773
  [63855]       chrY 57207346-57212230      + |   HAVANA     gene      4618
  [63856]       chrY 57212184-

### Create a query region of interest

In [2]:
query <- GenomicRanges::GRanges("chr21:10000000-10200000:+")
query

GRanges object with 1 range and 0 metadata columns:
      seqnames            ranges strand
         <Rle>         <IRanges>  <Rle>
  [1]    chr21 10000000-10200000      +
  -------
  seqinfo: 1 sequence from an unspecified genome; no seqlengths

### Check which genes overlap our query region

In [3]:
findOverlaps(query, genes_gencode_v26)

Hits object with 2 hits and 0 metadata columns:
      queryHits subjectHits
      <integer>   <integer>
  [1]         1       40078
  [2]         1       40079
  -------
  queryLength: 1 / subjectLength: 63856

### What happens if we ignore the strands?

In [4]:
findOverlaps(query, genes_gencode_v26, ignore.strand = TRUE) # default of ignore.strand is FALSE

Hits object with 4 hits and 0 metadata columns:
      queryHits subjectHits
      <integer>   <integer>
  [1]         1       40076
  [2]         1       40077
  [3]         1       40078
  [4]         1       40079
  -------
  queryLength: 1 / subjectLength: 63856

### Find the genes that overlap our query region

In [5]:
subjectHits(findOverlaps(query, genes_gencode_v26, ignore.strand = TRUE))

[1] 40076 40077 40078 40079

### Inspect them in more detail

In [6]:
genes_gencode_v26[subjectHits(findOverlaps(query, genes_gencode_v26, ignore.strand = TRUE))]

GRanges object with 4 ranges and 10 metadata columns:
      seqnames            ranges strand |   source     type     score     phase
         <Rle>         <IRanges>  <Rle> | <factor> <factor> <numeric> <integer>
  [1]    chr21 10028361-10029855      - |   HAVANA     gene      1495      <NA>
  [2]    chr21  9975017-10119309      - |   HAVANA     gene      1856      <NA>
  [3]    chr21 10122273-10129029      + |   HAVANA     gene      1942      <NA>
  [4]    chr21 10136419-10137004      + |   HAVANA     gene       586      <NA>
                gene_id                        gene_type      gene_name
            <character>                      <character>    <character>
  [1] ENSG00000280372.1                              TEC   bP-21201H5.2
  [2] ENSG00000270533.2 transcribed_processed_pseudogene   bP-21201H5.1
  [3] ENSG00000279851.2                          lincRNA CH507-216K13.2
  [4] ENSG00000279062.1           unprocessed_pseudogene CH507-216K13.1
            level          havana_

### Globally, how many genes overlap each other?

In [7]:
summary(countOverlaps(genes_gencode_v26))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    1.00    1.00    1.38    2.00   95.00 

### And if we ignore the strand?

In [8]:
summary(countOverlaps(genes_gencode_v26, ignore.strand = TRUE))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   2.000   1.944   2.000 100.000 

### What if we ask for at least 100 bp of overlap?

In [9]:
summary(countOverlaps(genes_gencode_v26, ignore.strand = TRUE, minoverlap = 100))
table(countOverlaps(genes_gencode_v26, ignore.strand = TRUE, minoverlap = 100))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   1.000   2.000   1.794   2.000  34.000 


    0     1     2     3     4     5     6     7     8     9    10    11    12 
 3138 27442 22716  6373  2136   948   467   242   116    72    37    32    23 
   13    14    15    16    17    18    20    21    22    23    25    26    27 
   14     9    11     8    10     7    11     7    12     2     2     1    15 
   28    34 
    4     1 

### Question about combining subjectHits and genes_gencdoe_v26

In [10]:
test <- genes_gencode_v26[subjectHits(findOverlaps(query, genes_gencode_v26, ignore.strand = TRUE))]
# pieces <- disjoint(c(query, test)); not sure if this works, but maybe? Seems faster to use bedtools

## Lets build our first SummarizedExperiment object

In [11]:
library("SummarizedExperiment")
?SummarizedExperiment

Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: DelayedArray

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following objects are masked from ‘package:Biobase’:

    anyMissing, rowMedians



Attaching package: ‘DelayedArray’


The following objects are masked from ‘package:matrixStats’:

    colMaxs, colMins, colRanges, rowMaxs, rowMins, rowRanges


The following objects are masked from ‘package:base’:

    aperm, apply, rowsum




RangedSummarizedExperiment-class {SummarizedExperiment},R Documentation
assays,"A list or SimpleList of matrix-like elements, or a matrix-like object (e.g. an ordinary matrix, a data frame, a DataFrame object from the S4Vectors package, a sparseMatrix derivative from the Matrix package, a DelayedMatrix object from the DelayedArray package, etc...). All elements of the list must have the same dimensions, and dimension names (if present) must be consistent across elements and with the row names of rowRanges and colData."
rowData,"A DataFrame object describing the rows. Row names, if present, become the row names of the SummarizedExperiment object. The number of rows of the DataFrame must equal the number of rows of the matrices in assays."
rowRanges,"A GRanges or GRangesList object describing the ranges of interest. Names, if present, become the row names of the SummarizedExperiment object. The length of the GRanges or GRangesList must equal the number of rows of the matrices in assays. If rowRanges is missing, a SummarizedExperiment instance is returned."
colData,"An optional DataFrame describing the samples. Row names, if present, become the column names of the RangedSummarizedExperiment."
metadata,An optional list of arbitrary content describing the overall experiment.
x,A RangedSummarizedExperiment object. The rowRanges setter will also accept a SummarizedExperiment object and will first coerce it to RangedSummarizedExperiment before it sets value on it.
...,Further arguments to be passed to or from other methods.
value,A GRanges or GRangesList object.
subset,"An expression which, when evaluated in the context of rowRanges(x), is a logical vector indicating elements or rows to keep: missing values are taken as false."
select,"An expression which, when evaluated in the context of colData(x), is a logical vector indicating elements or rows to keep: missing values are taken as false."


### From the example page:

In [12]:
nrows <- 200; ncols <- 6
counts <- matrix(runif(nrows * ncols, 1, 1e4), nrows)
rowRanges <- GRanges(rep(c("chr1", "chr2"), c(50, 150)),
                     IRanges(floor(runif(200, 1e5, 1e6)), width=100),
                     strand=sample(c("+", "-"), 200, TRUE),
                     feature_id=sprintf("ID%03d", 1:200))
colData <- DataFrame(Treatment=rep(c("ChIP", "Input"), 3),
                     row.names=LETTERS[1:6])
rse <- SummarizedExperiment(assays=SimpleList(counts=counts),
                            rowRanges=rowRanges, colData=colData)
rse
dim(rse)
dimnames(rse)
assayNames(rse)
head(assay(rse))

class: RangedSummarizedExperiment 
dim: 200 6 
metadata(0):
assays(1): counts
rownames: NULL
rowData names(1): feature_id
colnames(6): A B ... E F
colData names(1): Treatment

[1] 200   6

[[1]]
NULL

[[2]]
[1] "A" "B" "C" "D" "E" "F"

[1] "counts"

A,B,C,D,E,F
896.0646,5066.1967,4075.7451,5287.9803,5915.1151,2807.898
6445.7028,9319.9921,432.3704,1899.0602,1127.5161,7117.197
1605.2677,574.1738,9013.0374,4602.2937,5200.8837,2354.680
3195.1520,4034.5101,6206.0085,8566.9528,2047.0686,6295.844
4666.2822,7866.2613,6235.2263,708.3069,470.9615,5979.806
8649.6585,1462.4423,5013.9480,972.1192,2995.2052,9334.861


In [13]:
rowRanges(rse)
rowData(rse)  # same as 'mcols(rowRanges(rse))'
colData(rse)

GRanges object with 200 ranges and 1 metadata column:
        seqnames        ranges strand |  feature_id
           <Rle>     <IRanges>  <Rle> | <character>
    [1]     chr1 348245-348344      - |       ID001
    [2]     chr1 368465-368564      - |       ID002
    [3]     chr1 696524-696623      - |       ID003
    [4]     chr1 749724-749823      - |       ID004
    [5]     chr1 376001-376100      - |       ID005
    ...      ...           ...    ... .         ...
  [196]     chr2 821735-821834      + |       ID196
  [197]     chr2 552989-553088      + |       ID197
  [198]     chr2 454941-455040      - |       ID198
  [199]     chr2 556872-556971      - |       ID199
  [200]     chr2 853129-853228      + |       ID200
  -------
  seqinfo: 2 sequences from an unspecified genome; no seqlengths

DataFrame with 200 rows and 1 column
     feature_id
    <character>
1         ID001
2         ID002
3         ID003
4         ID004
5         ID005
...         ...
196       ID196
197       ID197
198       ID198
199       ID199
200       ID200

DataFrame with 6 rows and 1 column
    Treatment
  <character>
A        ChIP
B       Input
C        ChIP
D       Input
E        ChIP
F       Input

## Second half; advanced topics

```
## Optional:
BiocManager::install("iSEE")
BiocManager::install("recount")
BiocManager::install("spatialLIBD")
```

### iSEE (Interactive SummarizedExperiment Explorer)
- [Documentation](http://bioconductor.org/packages/release/bioc/html/iSEE.html)
- [vignette](http://bioconductor.org/packages/release/bioc/vignettes/iSEE/inst/doc/basic.html)

### Explore the rse object interactively

In [14]:
library("iSEE")
iSEE::iSEE(rse) 

Loading required package: SingleCellExperiment

Loading required package: shiny


Listening on http://127.0.0.1:3759



- This does not create a pop-up in juptyer-lab.
- Copy the url to a web browser. Loads very fast on my labtop (4 core, 16G RAM)
- This must be interrupted by hand or commented out.

## recount
- [Documentation](http://bioconductor.org/packages/release/bioc/html/recount.html)
- [Documentation](http://bioconductor.org/packages/release/bioc/vignettes/recount/inst/doc/recount-quickstart.html)

#### Obtain more RSE objects from recount2
[Leo's shinyapp](https://jhubiostatistics.shinyapps.io/recount/)

In [15]:
library("recount")
recount::download_study("SRP009615")
load(file.path("SRP009615", "rse_gene.Rdata"), verbose = TRUE)
rse_gene

Setting options('download.file.method.GEOquery'='auto')

Setting options('GEOquery.inmemory.gpl'=FALSE)

2020-05-16 09:54:01 downloading file rse_gene.Rdata to SRP009615



Loading objects:
  rse_gene


class: RangedSummarizedExperiment 
dim: 58037 12 
metadata(0):
assays(1): counts
rownames(58037): ENSG00000000003.14 ENSG00000000005.5 ...
  ENSG00000283698.1 ENSG00000283699.1
rowData names(3): gene_id bp_length symbol
colnames(12): SRR387777 SRR387778 ... SRR389083 SRR389084
colData names(21): project sample ... title characteristics

### Rename to avoid losing it in our next step

In [16]:
rse_gene_SRP009615 <- rse_gene

### [LIBD eQTL browser:](http://eqtl.brainseq.org/)
- [BrainSeq Phase II:](http://eqtl.brainseq.org/phase2/eqtl/)
- [Data files:](http://eqtl.brainseq.org/phase2/)

### BrainSeq Phase II
- This is a 290 Mb download! The download time is relatively quick.

In [17]:
download.file(
    paste0(
        "https://s3.us-east-2.amazonaws.com/", 
        "libd-brainseq2/rse_gene_unfiltered.Rdata"
    ),
    "rse_gene_unfiltered.Rdata",
    mode = "wb"
)
load("rse_gene_unfiltered.Rdata", verbose = TRUE)
rse_gene

Loading objects:
  rse_gene


class: RangedSummarizedExperiment 
dim: 58037 900 
metadata(0):
assays(2): counts rpkm
rownames(58037): ENSG00000223972.5 ENSG00000227232.5 ...
  ENSG00000210195.2 ENSG00000210196.2
rowData names(11): Length gencodeID ... gencodeTx passExprsCut
colnames(900): R10424 R12195 ... R5766 R5768
colData names(54): SAMPLE_ID FQCbasicStats ... snpPC9 snpPC10

## SingleCellExperiment
- [SingleCellExperiment Documentation](http://bioconductor.org/packages/release/bioc/html/SingleCellExperiment.html)
- [SingleCellExperiment Vignette](http://bioconductor.org/packages/release/bioc/vignettes/SingleCellExperiment/inst/doc/intro.html)
- [Tutoral: Orchestrating Single-Cell Analysis with Bioconductor](https://osca.bioconductor.org/)
- [SingleCellExperiment Manuscript](https://www.nature.com/articles/s41592-019-0654-x)

## spatialLIBD
- [spatialLIBD Documentation](http://bioconductor.org/packages/release/data/experiment/html/spatialLIBD.html)
- [spatialLIBD Vignette](http://bioconductor.org/packages/release/data/experiment/vignettes/spatialLIBD/inst/doc/spatialLIBD.html)

## Reproducibility information

In [18]:
#install.packages('sessioninfo')
print('Reproducibility information:')
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()

[1] "Reproducibility information:"


[1] "2020-05-16 09:54:59 EDT"

   user  system elapsed 
 24.431   3.711 123.052 

─ Session info ───────────────────────────────────────────────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.0 (2020-04-24)
 os       Arch Linux                  
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       America/New_York            
 date     2020-05-16                  

─ Packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────
 package              * version  date       lib source        
 acepack                1.4.1    2016-10-29 [1] CRAN (R 4.0.0)
 AnnotationDbi          1.50.0   2020-04-27 [1] Bioconductor  
 askpass                1.1      2019-01-13 [1] CRAN (R 4.0.0)
 assertthat             0.2.1    2019-03-21 [1] CRAN (R 4.0.0)
 backports              1.1.6    2020-04-05 [1] CRAN (R 